# **RAG Application** with LangChain and HuggingFace LLM

In [1]:
# Install the necessary packages
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
# Import necessary libraries
import os
from google.colab import userdata

### Initialize HuggingFace LLM

Model repo url: https://huggingface.co/mistralai/Mistral-7B-v0.1

In [3]:
from langchain_community.llms import HuggingFaceHub

# Initialize the HuggingFace llm
llm = HuggingFaceHub(
      repo_id = "mistralai/Mistral-7B-v0.1",
      model_kwargs = {"temperature":0.1, "max_length":500},
      huggingfacehub_api_token = userdata.get('HUGGINGFACE_API_KEY'))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


### Initialize Embedding Model

Model url: https://sbert.net/

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
  model_name="sentence-transformers/all-mpnet-base-v2"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Initialize Output Parser

In [5]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

### Load PDF Document

In [6]:
!pip install pypdf -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 5.1 MB/s eta 0:00:00


In [9]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/codeprolk.pdf")

docs = loader.load()

In [10]:
len(docs)

4

In [11]:
docs[0]

Document(metadata={'source': '/content/codeprolk.pdf', 'page': 0}, page_content="Introduction to CodePRO LK  \nCodePRO LK  is a dynamic educational platform that offers a diverse range of technology -\nrelated courses in Sinhala, aimed at empowering Sri Lankans with valuable skills in \nprogramming, data science, and machine learning. Founded by Dinesh Piyasamara  during the \nCOVID -19 pandemic, CodePRO LK addresses the growing need for accessible, high -quality \ntech education tailored to the local community.  \n \nFounding and Vision  \nOrigin and Motivation  \nThe inception of CodePRO LK was driven by the challenges posed by the COVID -19 pandemic, \nwhich highlighted the necessity for remote learning and digital skills. Recognizing this, Dinesh \nPiyasamara launched CodePRO LK to provide Sri Lankan students with the  tools and knowledge \nto thrive in a digitally -driven world, all through their native language.  \nVision and Mission  \n• Vision : To assist talented Sri Lankans i

### Split Documents into Chunks

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [22]:
len(splits)

20

### Create Vector Store and Retriever

In [23]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [24]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

### Define Prompt Template

In [34]:
from langchain.prompts import ChatPromptTemplate

# Define prompt template
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""

prompt=ChatPromptTemplate.from_template(template)

In [35]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'))])

### Chain Retriever and Prompt Template with LLM

In [36]:
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

#### Invoke RAG Chain with Example Questions

In [37]:
response = chain.invoke("who is codeprolk?")
print(response)

Human: 
Answer this question using the provided context only.

who is codeprolk?

Context:
[Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Partnerships and Collaborations  \nCodePRO LK is exploring partnerships with educational institutions, tech companies, and \nindustry experts to enrich its content and provide learners with access to a broader range of \nresources and opportunities. These collaborations aim to bridge the gap be tween education and \nindustry, ensuring that learners are well -prepared for real -world challenges.'), Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Partnerships and Collaborations  \nCodePRO LK is exploring partnerships with educational institutions, tech companies, and \nindustry experts to enrich its content and provide learners with access to a broader range of \nresources and opportunities. These collaborations aim to bridge the gap be tween education and \nindustry, ensuring that lea

In [38]:
response = chain.invoke("what is generative ai?")
print(response)

Human: 
Answer this question using the provided context only.

what is generative ai?

Context:
[Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Enhanced Learning Tools  \nThe platform plans to integrate more interactive and adaptive learning tools to personalize the \nlearning experience further. This includes AI -driven recommendations, gamified learning \nexperiences, and enhanced assessment tools to track and improve learner progress effectively.  \nCommunity Engagement and Events'), Document(metadata={'page': 3, 'source': '/content/codeprolk.pdf'}, page_content='Enhanced Learning Tools  \nThe platform plans to integrate more interactive and adaptive learning tools to personalize the \nlearning experience further. This includes AI -driven recommendations, gamified learning \nexperiences, and enhanced assessment tools to track and improve learner progress effectively.  \nCommunity Engagement and Events'), Document(metadata={'page': 1, 'source': '/con